In [74]:
import pandas as pd
import duckdb
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score



In [42]:
# Initialize a connection and create a persistent database
# Worth noting that due to the workflow I'm using, the database was/should be created externally, and then built into the Docker container
# This way, the raw database files are saved locally, but file size won't grow exponentially
con = duckdb.connect("files/titanic.duckdb")


In [43]:
# Using this to DROP and Recreate train_raw, ensuring a fresh process
con.sql("DROP TABLE train_raw;")
con.sql("DROP TABLE test_raw;")
con.sql("CREATE TABLE train_raw AS SELECT * FROM 'files/train.csv'")
con.sql("CREATE TABLE test_raw AS SELECT * FROM 'files/test.csv'")

In [44]:
# Create working tables
con.sql("DROP TABLE train;")
con.sql("DROP TABLE test;")
con.sql("CREATE TABLE train AS SELECT * FROM train_raw")
con.sql("CREATE TABLE test AS SELECT * FROM test_raw")

In [45]:
# Verify the proper tables are loaded
con.sql("SELECT * FROM duckdb_tables()")

┌───────────────┬──────────────┬─────────────┬────────────┬────────────┬───────────┬─────────┬───────────────────────┬──────────┬───────────┬─────────────────┬────────────────┬──────────────┬─────────────┬────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ database_name │ database_oid │ schema_name │ schema_oid │ table_name │ table_oid │ comment │         tags          │ internal │ temporary │ has_primary_key │ estimated_size │ column_count │ index_count │ check_constraint_count │                                                                                                      sql                                                                                                       │
│    varchar    │    int64     │   varchar   │   int64    │  varchar   │   int64   │ varchar │ map(varchar, varchar) │ b

In [59]:
# Generate summary statistics
con.sql("SUMMARIZE train")
#con.sql("SUMMARIZE test")

┌─────────────┬─────────────┬─────────┬─────────┬───────────────┬─────────────────────┬─────────────────────┬─────────┬───────────────────┬───────────────────┬───────┬─────────────────┐
│ column_name │ column_type │   min   │   max   │ approx_unique │         avg         │         std         │   q25   │        q50        │        q75        │ count │ null_percentage │
│   varchar   │   varchar   │ varchar │ varchar │     int64     │       varchar       │       varchar       │ varchar │      varchar      │      varchar      │ int64 │  decimal(9,2)   │
├─────────────┼─────────────┼─────────┼─────────┼───────────────┼─────────────────────┼─────────────────────┼─────────┼───────────────────┼───────────────────┼───────┼─────────────────┤
│ PassengerId │ BIGINT      │ 1       │ 891     │          1051 │ 446.0               │ 257.3538420152301   │ 223     │ 446               │ 669               │   891 │            0.00 │
│ Survived    │ BIGINT      │ 0       │ 1       │             2 │ 0.38

In [47]:
# Examine Nulls for the Age, Cabin, and Embarked columns (do this for test as well)
con.sql("SELECT * FROM train WHERE Age IS NULL") # Seems to make the most sense to use the average age here
con.sql("SELECT * FROM train WHERE Cabin IS NULL") # Seems likely Cabins not as strictly recorded for lower class guests, probably unnecessary for model
con.sql("SELECT * FROM train WHERE Embarked IS NULL") # This only comprises 2 records and it's unclear if they made it on in the first place, not a high enough percentage of 1st class survivors to consider keeping

┌─────────────┬──────────┬────────┬───────────────────────────────────────────┬─────────┬────────┬───────┬───────┬─────────┬────────┬─────────┬──────────┐
│ PassengerId │ Survived │ Pclass │                   Name                    │   Sex   │  Age   │ SibSp │ Parch │ Ticket  │  Fare  │  Cabin  │ Embarked │
│    int64    │  int64   │ int64  │                  varchar                  │ varchar │ double │ int64 │ int64 │ varchar │ double │ varchar │ varchar  │
├─────────────┼──────────┼────────┼───────────────────────────────────────────┼─────────┼────────┼───────┼───────┼─────────┼────────┼─────────┼──────────┤
│          62 │        1 │      1 │ Icard, Miss. Amelie                       │ female  │   38.0 │     0 │     0 │ 113572  │   80.0 │ B28     │ NULL     │
│         830 │        1 │      1 │ Stone, Mrs. George Nelson (Martha Evelyn) │ female  │   62.0 │     0 │     0 │ 113572  │   80.0 │ B28     │ NULL     │
└─────────────┴──────────┴────────┴───────────────────────────────────

In [48]:
# Update the Age column, replace NULL values with the average Age
con.sql("""UPDATE train AS train_clean
        SET Age = (
            SELECT
                avg(raw.Age) AS cleanAge
            FROM train as raw
            WHERE raw.Age IS NOT NULL
        )
        WHERE Age IS NULL""")

In [49]:
# Update the Sex column, change the VARCHAR type to BOOLEAN
con.sql("""ALTER TABLE train ALTER Sex 
        SET DATA TYPE BOOLEAN USING CASE
            WHEN Sex = 'female' THEN 1 ELSE 0 END
        """)

In [50]:
# Update the Age column in the test dataset
con.sql("""UPDATE test AS test_clean
        SET Age = (
            SELECT
                avg(raw.Age) AS cleanAge
            FROM test as raw
            WHERE raw.Age IS NOT NULL
        )
        WHERE Age IS NULL""")

In [51]:
# Update the Sex column, change the VARCHAR type to BOOLEAN
con.sql("""ALTER TABLE test ALTER Sex 
        SET DATA TYPE BOOLEAN USING CASE
            WHEN Sex = 'female' THEN 1 ELSE 0 END
        """)

In [ ]:
# Remove the Name, Cabin, Embarked, Fare, and Ticket columns
con.sql("ALTER TABLE train DROP Name") # Has to be numeric data
con.sql("ALTER TABLE train DROP Cabin")
con.sql("ALTER TABLE train DROP Embarked")
con.sql("ALTER TABLE train DROP Fare") # Dropping because there are nulls in the test file
con.sql("ALTER TABLE train DROP Ticket") # Dropping because of inconsistent values


In [ ]:
# Remove the Name, Cabin, Embarked, Fare, and Ticket columns
con.sql("ALTER TABLE test DROP Name") # Has to be numeric data
con.sql("ALTER TABLE test DROP Cabin")
con.sql("ALTER TABLE test DROP Embarked")
con.sql("ALTER TABLE test DROP Fare") # Dropping because there are nulls in the test file
con.sql("ALTER TABLE test DROP Ticket") # Dropping because of inconsistent values



In [60]:
# Creating dataframes for testing/training, I'll be using sklearn here, which needs both
train = con.sql("SELECT * FROM train").df()
test = con.sql("SELECT * FROM test").df()

In [66]:
# Create features and target
X = train.drop("Survived", axis = 1).values
y = train["Survived"].values

In [69]:
# Initialize Regression object and split data
logreg = LogisticRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 123)

In [70]:
# Fit and predict
logreg.fit(X_train, y_train)

/usr/local/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [72]:
# Predict
y_pred = logreg.predict(X_test)

In [75]:
y_pred_probs = logreg.predict_proba(X_test)[:, 1]
print(roc_auc_score(y_test, y_pred_probs))

0.8368699526594263
